## Formulação matemática do problema da Grade Horária na UFPR Campus Pontal (Timetabling)


### Objetivo

O objetivo desta modelagem é minimizar a quantidade de vezes que os professores da UFPR Campus Pontal vão ao campus.
Vamos levar em conta dados originais obtidos em contato com a universidade e restrições reais.


### Parâmetros

Para modelar, temos as variáveis: 

* **T**: conjunto de turmas (8 turmas em LCE)
* **P**: conjunto de professores (16 professores em LCE)
* **H**: conjunto de horários das aulas (1 ou 2 - primeiro ou segundo horário)
* **D**: conjunto de dias da semana que ocorrem aulas (varia de segunda a sexta)

Total de combinações: $8 * 16 * 2 * 5 = 1280$.


### Variáveis de Decisão

As variáveis $x_{p,t,d,h}$ são inteiras e binárias, que definem se o professor $p$ irá (1) ou não (0) ministrar aula para a turma $t \in T$ no dia $d \in D$ e no horário $h \in H$.


### Problema de Otimização

$$
\begin{align}
    \text{minimizar \ \ \ \ \ } & \sum_{d=1}^{5}x_{p,t,d,h} \\
    \text{sujeito a \ \ \ \ \ } & \sum_{p=1}^{16}x_{p,t,d,h} \leq 1 \\
                                & \sum_{t=1}^{8}x_{p,t,d,h} \leq 1 \\
                                & \sum_{p=1}^{16}\sum_{h=1}^{2}x_{p,t,d,h} = HT_{t,d} \\
                                & \sum_{d=1}^{5}\sum_{h=1}^{2}x_{p,t,d,h} = R_{p,t} \\
                                & \sum_{t \in T_1}\sum_{p \in P_1}x_{p,t,d,h} = 2 \\
                                & \sum_{h=1}^{}x_{ptdh} \leq 6
\end{align}
$$

onde:

- $H$:
- $T_{t,d}$:
- $R_{p,t}$:
- $T_1$: subconjunto das turmas $T$ com aulas comuns às três habilitações
- $P_1$: subconjunto dos professores $P$ que ministram aulas nas disciplinas comuns

### Explicação das Restrições

1. Cada combinação de turma, horário e dia da semana terá somente 1 professor alocado
2. Cada combinação de professor, horário e dia da semana terá somente 1 turma sendo ministrada
3. 
4.
5.

In [26]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [31]:
def constroi_lista(df:pd.DataFrame, coluna:str):
    lista = list(df[df['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])][coluna].dropna().unique())
    return lista

In [27]:
grade_compilada = pd.read_csv('dados/grade_compilada.csv', sep=';')
grade_compilada_tratado = pd.read_csv('dados/grade_compilada_tratado.csv', sep=',')
hora_aula_materia = pd.read_csv('dados/hora_aula_materia.csv', sep=',')
materias_comuns = pd.read_csv('dados/materias_comuns.csv', sep=',')
professores_materias = pd.read_csv('dados/professores_materias.csv', sep=',')

In [40]:
grade_compilada

,Campus,Turma,Período do Dia,Parte do Horário,Semestre,Dia,Tipo de Curso,Tipo de Turma,Aula,prof,Observação
0,MIRASOL,EAQ,MANHÃ,1.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUÇÃO A AQUICULTURA,LAGREZE,NaN
1,MIRASOL,EAQ,MANHÃ,2.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUÇÃO A AQUICULTURA,LAGREZE,NaN
2,MIRASOL,EAQ,MANHÃ,3.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
3,MIRASOL,EAQ,MANHÃ,4.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
4,MIRASOL,EAQ,MANHÃ,1.0,1.0,TERÇA,MATUTINO,INTEGRAL MT,INTRODUÇÃO A QUALIDADE,SACHSIDA,NaN
...,...,...,...,...,...,...,...,...,...,...,...
990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
grade_compilada_tratado.head()

,Campus,Curso,Periodo_dia,Horario,Semestre,Dia,Periodo_Aula,Tipo_Curso,Materia,Professor,Observacao
0,MIRASOL,EAQ,MANHA,1.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN
1,MIRASOL,EAQ,MANHA,2.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN
2,MIRASOL,EAQ,MANHA,3.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
3,MIRASOL,EAQ,MANHA,4.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
4,MIRASOL,EAQ,MANHA,1.0,1.0,TERCA,MATUTINO,INTEGRAL MT,INTRODUCAO A QUALIDADE,SACHSIDA,NaN


In [41]:
grade_compilada_tratado['Semestre'].value_counts()

Semestre
5.0    138
7.0    137
3.0    120
1.0    111
9.0     86
Name: count, dtype: int64

In [4]:
hora_aula_materia.head()

,Materia,Curso,Campus
0,ALGAS NOCIVAS E TOXINASOPT,EAQ,4
1,ALGEBRA LINEAR,OCEANO,3
2,ALGICULTURA,EAQ,4
3,ANALISE CEM324,LCEMATEMATICA,4
4,ANALISE COMPUTACIONAL DE ESTRUTURAS,ECV,4


In [15]:
hora_aula_materia[hora_aula_materia['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])]['Materia'].unique()

array(['ANALISE CEM324', 'CALCULO INTEGRAL LCE131',
       'COMPUTACAO II LCE133', 'DIDATICA DAS CIENCIAS CEM334',
       'DIVULGACAO CIENTIFICA CEM326',
       'EAD HISTORIA FILOSOFIA E ENSINO LCE135',
       'EAD METODOLOGIA CIENTIFICA LCE136', 'ESTAGIO CIENCIAS I CEM336',
       'ESTAGIO FISICA I CEM363', 'ESTAGIO FISICA II CEM399',
       'ESTAGIO MATEMATICA I CEM343', 'ESTAGIO QUIMICA I CEM364',
       'ETICA E EDUCACAO LCE134', 'FISICA EXPERIMENTAL I CEM347',
       'FISICA EXPERIMENTAL III CEM349', 'FISICA I PP001',
       'FISICA III PP005', 'FISICA MODERNA I CEM352',
       'FISICOQUIMICA CEM365', 'FUNDAMENTOS DA EDUCACAO LCE113',
       'FUNDAMENTOS DA EXTENSAO PP027', 'GEOMETRIA ANALITICA PP002',
       'GEOMETRIA E CONSTRUCOES CEM319',
       'HISTORIA FILOSOFIA E ENSINO LCE135',
       'MATEMATICA ELEMENTAR LCE111', 'MATEMATICA V CEM306',
       'METODOLOGIA CIENTIFICA LCE136',
       'OPTATIVA I TOPICOS ESPECIAIS I LCE911',
       'OPTATIVA II TOPICOS ESPECIAIS II LCE912'

In [5]:
materias_comuns.head()

,Professor,Campus,Materia
0,VALDIR,MIRASOL,PRATICA PEDAGOGICA DO ENSINO CEM335
1,JEINNI,MIRASOL,DIDATICA DAS CIENCIAS CEM334
2,JEINNI,MIRASOL,ESTAGIO CIENCIAS I CEM336
3,ELIANE,MIRASOL,DIVULGACAO CIENTIFICA CEM326


In [6]:
professores_materias.head()

,Professor,Campus,Curso,Tipo_Curso,Materia
0,LAGREZE,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A AQUICULTURA
1,CENDON,MIRASOL,EAQ,INTEGRAL MT,PROBABILIDADE
2,SACHSIDA,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A QUALIDADE
3,RODOLFO,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A QUALIDADE
4,LUCIANA,MIRASOL,EAQ,INTEGRAL MT,GEOMETRIA ANALITICA


In [7]:
professores_materias['Curso'].value_counts()

Curso
OCEANO           49
ECV              48
EAS              39
EAQ              36
LCE              21
LCEFISICA        12
LCEMATEMATICA    10
LCEQUIMICA        9
Name: count, dtype: int64

In [12]:
professores_lce = list(professores_materias[professores_materias['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])]['Professor'].unique())
#professores_materias['Campus'] == 'MIRASOL'
professores_lce

['GUILHERME',
 'LUCIANA',
 'BATISTA',
 'SELMA',
 'ELIANE',
 'BACALHAU',
 'TALAL',
 'ROGERIO',
 'JEINNI',
 'EMIR',
 'ELIZABETE',
 'VALDIR',
 'ALEX',
 'PEDRO',
 'CASSIO',
 'BORGES',
 'HARUMI']

In [ ]:
import gurobipy as gp
from gurobipy import GRB


# Inicialização do modelo
model = gp.Model("Alocação_Professores")

# Define os conjuntos
professores = constroi_lista(df = grade_compilada_tratado, coluna = 'Professor') # 17 (atual) 16 (artigo)
turmas = constroi_lista(df = grade_compilada_tratado, coluna = 'Semestre') # 5 (atual) 8 (artigo)
# atualmente não estamos segmentando o 5º e 7º em qui, fis e mat e temos o 9º que não estava no artigo
dias_semana = constroi_lista(df = grade_compilada_tratado, coluna = 'Dia') # range(1, 6)
horarios = constroi_lista(df = grade_compilada_tratado, coluna = 'Periodo_dia') # range(1, 3)
professores_comuns = 
turmas_comuns = 

# Cria as variáveis de decisão
x = model.addVars(professores, turmas, dias_semana, horarios, vtype=GRB.BINARY, name="x")

# Define a função objetivo
model.setObjective(gp.quicksum(x[p,t,d,h] for p in professores for t in turmas for d in dias_semana for h in horarios), GRB.MINIMIZE)

# Restrição 2 - Cada turma tem no máximo um professor em um horário específico
for t in turmas:
    for d in dias_semana:
        for h in horarios:
            model.addConstr(gp.quicksum(x[p,t,d,h] for p in professores) <= 1)

# Restrição 3 - Cada professor tem no máximo uma aula em um horário específico
for p in professores:
    for d in dias_semana:
        for h in horarios:
            model.addConstr(gp.quicksum(x[p,t,d,h] for t in turmas) <= 1)

# Restrição 4
# "relação de disciplinas de cada turma"
for t in turmas:
    for d in dias_semana:
        model.addConstr(gp.quicksum(x[p,t,d,h] for p in professores for h in dias_semana) == 2) # HT_t,d

# Restrição 5
# "associa cada disciplina ao professora que ira lecionar cada uma delas"
for p in professores:
    for t in turmas:
        model.addConstr(gp.quicksum(x[p,t,d,h] for d in dias_semana for h in horarios) == 4) # R_p,t

# Restrição 6
# T1 ⊂ T (subconj. das turmas com aulas comuns) e P1 ⊂ P (subconj. dos professores das aulas comuns)
for p in professores_comuns:
    for t in turmas_comuns:
        model.addConstr(gp.quicksum(x[p,t,d,h] for d in dias_semana for h in horarios) == 2)

# Restrição 7
# Garante que as aulas sejam alocadas nas 6 salas de aulas disponíveis
# Isto é, a quantidade de aulas a cada horário deve ser no máximo 6, pois esse é o limite de salas
for p in professores:
    for t in turmas:
        for d in dias_semana:
            model.addConstr(gp.quicksum(x[p,t,d,h] for h in horarios) <= 6)


# Resolve o modelo
model.optimize()

# Imprime a solução
if model.Status == GRB.OPTIMAL:
    for p, t, d, h in x:
        if x[p,t,d,h].X > 0:
            print(f"Professor {p} dá aula na disciplina {d} no turno {t} e no dia {h}")
else:
    print('O modelo não foi resolvido')